In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from mpl_toolkits.mplot3d import Axes3D
from numba import njit

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6), 'figure.dpi': 130,
                     'axes.titlesize': 'x-large', 'axes.labelsize': 'large',
                     'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 3: Dynamic Models
## Chapter 3.5: Robust Control and Ambiguity

### Table of Contents
1.  [From Risk to Ambiguity](#1.-From-Risk-to-Ambiguity)
    *   [1.1 Knightian Uncertainty](#1.1-Knightian-Uncertainty)
    *   [1.2 Axiomatic Foundations: Why Max-Min?](#1.2-Axiomatic-Foundations:-Why-Max-Min?)
2.  [The Hansen-Sargent Robust Control Framework](#2.-The-Hansen-Sargent-Robust-Control-Framework)
    *   [2.1 The Multiplier Problem and the Robust Bellman Equation](#2.1-The-Multiplier-Problem-and-the-Robust-Bellman-Equation)
    *   [2.2 The Worst-Case Belief](#2.2-The-Worst-Case-Belief)
3.  [Application: Robust Precautionary Savings](#3.-Application:-Robust-Precautionary-Savings)
4.  [Alternative: Smooth Ambiguity Aversion (KMM)](#4.-Alternative:-Smooth-Ambiguity-Aversion-(KMM))
5.  [Chapter Summary](#5.-Chapter-Summary)
6.  [Exercises](#6.-Exercises)

### 1. From Risk to Ambiguity

#### 1.1 Knightian Uncertainty
The models we have studied so far assume the agent knows the true probability distribution governing random shocks. This is the world of **risk**. However, in many real-world situations, agents face a er form of uncertainty, often called **Knightian uncertainty** or **ambiguity**, where the true probability model itself is unknown. Agents may have a baseline model they believe is a good approximation, but they fear it might be misspecified in ways they cannot precisely quantify.

**Robust control theory**, pioneered in macroeconomics by Lars Peter Hansen and Thomas Sargent, provides a framework for modeling decision-making in this environment. It models an agent who is not only risk-averse but also **ambiguity-averse**. Such an agent chooses policies that are **robust**—that is, policies that perform reasonably well across a whole set of plausible alternative models, not just the single baseline model they think is most likely.

#### 1.2 Axiomatic Foundations: Why Max-Min?
The standard expected utility model of von Neumann and Morgenstern is derived from a set of axioms about preference, including **completeness** (any two lotteries can be compared) and **transitivity**. However, the key axiom is the **independence axiom**, which states that if an agent prefers lottery A to lottery B, they must also prefer a mix of A and C to a mix of B and C.

The famous **Ellsberg paradox** showed that people systematically violate this axiom in the presence of ambiguity. This led to the development of alternative axiomatic systems. The model of **Gilboa and Schmeidler (1989)** relaxes the independence axiom and replaces it with a weaker **uncertainty aversion axiom**. This new set of axioms leads not to a single expected utility representation, but to a **max-min expected utility** representation, where the agent has a set of plausible probability distributions and evaluates a choice by the worst possible expected utility across that set. This provides the theoretical justification for the max-min structure of robust control.

### 2. The Hansen-Sargent Robust Control Framework
The core idea is to frame the agent's problem as a two-player game against a malevolent "evil agent" or "nature." The agent chooses a policy (a control law), and then nature, seeking to minimize the agent's welfare, chooses the worst possible realization of the model from a set of plausible alternatives. The agent, anticipating this, chooses the policy that gives the best outcome in the worst-case scenario. This is a **minimax** strategy.

#### 2.1 The Multiplier Problem and the Robust Bellman Equation
Let the agent have a baseline belief about the shock density, $\phi_0(w)$. They fear the true density $\phi(w)$ might be different. The set of alternative models is constrained by the **relative entropy**, $R(\phi || \phi_0) = \int \phi(w) \ln(\frac{\phi(w)}{\phi_0(w)}) dw$, which measures the statistical "distance" between the two distributions. The robust Bellman equation is a constrained optimization problem:
$$ V(s) = \max_{a} \min_{\phi} \left\{ r(s, a) + \beta \left( \int V(s') \phi(w) dw \right) \right\} \quad \text{s.t.} \quad R(\phi || \phi_0) \le \eta $$
This can be solved using a Lagrange multiplier $\theta$ and converted into an unconstrained saddle-point problem. Solving the inner minimization problem (nature's problem) leads to a more tractable form known as the **recursive entropy Bellman equation**:
$$ V(s) = \max_{a} \left\{ r(s, a) - \beta \theta \ln E_{\phi_0}[e^{-V(s')/\theta}] \right\} $$ 
The parameter $\theta$ is the **robustness parameter**. A small $\theta$ means the agent is highly ambiguity-averse. As $\theta \to \infty$, the agent becomes ambiguity-neutral, and we recover the standard Bellman equation.

#### 2.2 The Worst-Case Belief
A key result from the theory is that the robust agent behaves *as if* they are an expected utility maximizer, but with a **pessimistically distorted** or **worst-case** belief. For a given continuation value function $V(s')$, the worst-case probability of state $i$ is:
$$ p_{wc}(i) = \frac{p_0(i) \exp(-V_i / \theta)}{\sum_j p_0(j) \exp(-V_j / \theta)} $$
The evil agent slants the probabilities towards states that deliver the lowest continuation utility. The robustness parameter $\theta$ controls the degree of this slant.

### 3. Application: Robust Precautionary Savings
We now solve a simple two-period savings problem to see how ambiguity aversion affects precautionary savings. An agent has wealth $W$ today. Tomorrow, their income can be high ($y_H$) or low ($y_L$) with baseline probabilities $p_0$ and $1-p_0$. They want to choose consumption today ($c_0$) to maximize lifetime utility. A robust agent fears the probability of the low-income state might be higher than their baseline belief.

In [ ]:
sec("Robust Precautionary Savings")
# --- Model Parameters ---
W0, R, BETA, GAMMA = 100, 1.02, 0.96, 2.0
Y_H, Y_L, P0_H = 15.0, 5.0, 0.5

@njit
def u(c, gamma=GAMMA): return (c**(1 - gamma)) / (1 - gamma)

def solve_savings_problem(theta=np.inf):
    """Solves for optimal c0 given a robustness parameter theta."""
    def objective(c0):
        # Payoffs in the two states tomorrow
        c1_H = R * (W0 - c0) + Y_H
        c1_L = R * (W0 - c0) + Y_L
        # Utility from consumption tomorrow
        u_outcomes = np.array([u(c1_H), u(c1_L)])
        
        if theta == np.inf: # Standard EU case
            EV = P0_H * u_outcomes[0] + (1 - P0_H) * u_outcomes[1]
            return -(u(c0) + BETA * EV)
        else: # Robust case
            # Use the risk-sensitive operator
            p_baseline = np.array([P0_H, 1 - P0_H])
            distorted_factors = p_baseline * np.exp(u_outcomes / theta) # Note: V = u, so -V/theta -> u/theta for max problem
            risk_sensitive_EV = theta * np.log(np.sum(distorted_factors))
            return -(u(c0) + BETA * risk_sensitive_EV)
    
    res = minimize_scalar(objective, bounds=(0.1, W0-0.1), method='bounded')
    return res.x

# --- Solve and Compare ---
c0_eu = solve_savings_problem()
c0_robust_low_theta = solve_savings_problem(theta=1.0) # High ambiguity aversion
c0_robust_high_theta = solve_savings_problem(theta=50.0) # Low ambiguity aversion

note(f"Optimal Consumption (Standard EU):         c0 = {c0_eu:.2f}")
note(f"Optimal Consumption (Robust, high aversion): c0 = {c0_robust_low_theta:.2f}")
note(f"Optimal Consumption (Robust, low aversion):  c0 = {c0_robust_high_theta:.2f}")
note("The robust agent consumes less (saves more) than the standard agent. This extra saving is a buffer against their fear of model misspecification and is known as 'ambiguity-driven precautionary savings'.")

### 4. Alternative: Smooth Ambiguity Aversion (KMM)
The Hansen-Sargent model has a "kink" in the utility function at the reference model, which can sometimes lead to counter-intuitive behavior. An influential alternative is the **smooth ambiguity aversion** model of **Klibanoff, Marinacci, and Mukerji (2005)**.

The KMM model separates the agent's attitude towards risk from their attitude towards ambiguity. The agent has a utility function $u$ over consumption (capturing risk aversion) and a separate increasing and concave function $\phi$ over expected utilities, which captures ambiguity aversion. The agent evaluates a choice by:
$$ V_{KMM} = E_\mu [\phi(E_p[u(c)])] $$
Here, the agent has a subjective prior $\mu$ over a set of possible probability models $p$. The function $\phi$ determines how they feel about the uncertainty over which model is correct. A more concave $\phi$ implies higher ambiguity aversion. This framework leads to a smoother objective function and can better distinguish between risk and ambiguity in empirical applications.

### 5. Chapter Summary
- **Risk vs. Ambiguity:** Risk refers to uncertainty with known probabilities; ambiguity refers to uncertainty where the probability model itself is unknown. Standard expected utility theory fails to capture aversion to ambiguity, as shown by the Ellsberg paradox.
- **Robust Control:** The Hansen-Sargent framework models ambiguity aversion by having an agent play a game against a malevolent nature who chooses the worst possible model from a constrained set. The agent chooses a policy that is robust to this worst-case misspecification.
- **Robust Bellman Equation:** This game leads to a modified Bellman equation featuring a **risk-sensitive operator**, $- \theta \ln E[e^{-V/\theta}]$. The parameter $\theta$ governs the agent's ambiguity aversion.
- **Worst-Case Beliefs:** A robust agent behaves like an EU maximizer with pessimistically distorted beliefs, overweighting the probability of bad outcomes.
- **Economic Implications:** Ambiguity aversion provides a powerful explanation for observed phenomena that are puzzling under standard models, such as high equity premia and strong precautionary savings motives.
- **Alternative Models:** The smooth ambiguity model of KMM provides a related but distinct framework that separates risk aversion from ambiguity aversion.

### 6. Exercises

1.  **Risk vs. Ambiguity:** Explain the difference between risk and ambiguity using the example of a clinical trial for a new drug. What aspects involve risk, and what aspects involve ambiguity?

2.  **Risk Aversion and Ambiguity Aversion:** In the robust savings problem, re-run the analysis with a higher risk aversion parameter, `GAMMA = 5.0`. Does the precautionary savings motive (the gap between $c_0^{EU}$ and $c_0^{Robust}$) increase or decrease? Explain the interaction between the two aversions.

3.  **The KMM Model:** The certainty equivalent for the KMM model with CARA-Normal assumptions takes the form $CE = E[W] - \frac{1}{2}\rho Var(W) - \frac{1}{2}\eta Var(E[W|p])$. The first two terms are standard risk aversion. The third term is the ambiguity premium, where $\eta$ is the ambiguity aversion parameter. How does this decomposition differ from the Hansen-Sargent model?

4.  **Robust Social Planner:** Consider a social planner setting carbon tax policy. The planner has a model of the economic damages from climate change, but is uncertain about the 'tail risk' (the probability of a catastrophic outcome). How would a robust social planner's optimal carbon tax differ from a standard expected-cost-benefit planner? Would it be higher or lower? Why?

5.  **Worst-Case Probability:** For the savings problem, calculate and plot the worst-case probability of the low-income state as a function of the agent's savings choice, $c_0$. Does the agent's fear (the worst-case probability) get worse or better as they save more? Explain.